# Выбор локации для скважины

Перед добывающей компанией стоит вопрос, где бурить новую скважину.

В нашем распоряжении имеются пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Возможную прибыль и риски необходимо проанализировать техникой *Bootstrap.*

Шаги для выбора локации:

 - поиск месторождений в избранном регионе, определение признаков для каждого;
 - построение модели и оценка объёма запасов;
 - выбор месторождений с самыми высокими оценками значений (количество зависит от бюджета компании и стоимости разработки одной скважины);
 - определение прибыли как суммы прибылей оторанных месторождений.

## Загрузка и подготовка данных

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from tqdm import tqdm

In [2]:
seed = 6
state = np.random.RandomState(seed)

In [3]:
try:
    df_0 = pd.read_csv('geo_data_0.csv')
except:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')

In [4]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
try:
    df_1 = pd.read_csv('geo_data_1.csv')
except:
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')

In [7]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
try:
    df_2 = pd.read_csv('geo_data_2.csv')
except:
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [10]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [11]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В каждой таблице по 5 столбцов с различными типами данных. Согласно документации к данным:
 - `id` — уникальный идентификатор скважины;
 - `f0`, `f1`, `f2` —  три признака точек;
 - `product` — объём запасов в скважине (тыс. баррелей).
 
Пропусков в данных не обнаружено.

Информация об идентификаторе скважины не потребуется для дальнейшего анализа, поэтому удалим из таблиц столбец `id`.

In [12]:
for df in [df_0, df_1, df_2]:
    df.drop('id', axis=1, inplace=True)

In [13]:
for df in [df_0, df_1, df_2]:
    display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


None

Перейдём к обучению и проверке модели.

## Обучение и проверка модели

In [14]:
# функция подготовки данных, обучения модели и предсказания результатов
# входной параметр - датасет
# возвращаемые значения - датасет с предсказанными значениями, правильными
# ответами и PMSE модели

def prepare_and_predict(df):
    # выделение признаков и целевого признака
    features = df.drop('product', axis=1)
    target = df['product']

    # разбиение на обучающую и валидационную выборки
    (features_train, features_valid,
     target_train, target_valid) = train_test_split(features, target,
                                                    test_size=0.25,
                                                    random_state=seed)
    # масштабирование данных
    scaler = StandardScaler()
    scaler.fit(features_train)
    for f in [features_train, features_valid]:
        f = scaler.transform(f)

    # обучение модели и предсказание
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    # подготовка возвращаемых значений
    results = pd.DataFrame(data={'predicted':predictions_valid,
                                 'target':target_valid},
                                 columns=['predicted', 'target'])
    rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5
    return results, rmse

In [15]:
results_on_valid = []

In [16]:
i = 1
for df in [df_0, df_1, df_2]:
    results, rmse = prepare_and_predict(df)
    results_on_valid.append(results)
    print(f'Регион {i}:')
    print(f'Сред. запас: {results["predicted"].mean()}')
    print(f'RMSE модели: {rmse}')
    print()
    i += 1

Регион 1:
Сред. запас: 92.46866515483404
RMSE модели: 37.50092951247967

Регион 2:
Сред. запас: 68.77217858390492
RMSE модели: 0.8899637258179824

Регион 3:
Сред. запас: 94.68357707424651
RMSE модели: 40.05955871704578



Точнее всего запасы сырья модель предсказала для второго региона - корень из среднеквадратичной ошибки оказался меньше 1 тыс. баррелей (RMSE = 0.89). Далее идут первый и третий регионы со значениями RMSE равными соответственно 37.5 и 40.1.

## Подготовка к расчёту прибыли

In [17]:
BUDGET = 10_000_000_000
INCOME_PER_BARREL = 450_000

RESEARCH_WELLS = 500
SELECTED_WELLS = 200
SAMPLES = 1000

In [18]:
region_profit_barrels = BUDGET / INCOME_PER_BARREL
well_profit_barrels = region_profit_barrels / SELECTED_WELLS
print('Необходимый объём сырья для безубыточной разработки:')
print(f' на регион - {region_profit_barrels:.1f} тыс.баррелей')
print(f' на скважину - {well_profit_barrels:.1f} тыс.баррелей')

Необходимый объём сырья для безубыточной разработки:
 на регион - 22222.2 тыс.баррелей
 на скважину - 111.1 тыс.баррелей


Видим, что запасы в скважинах во всех регионах в среднем меньше, чем необходимо для безубыточной разработки новой скважины.

## Расчёт прибыли и рисков 

In [19]:
# функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(target, predicted):
    predicted_sorted = predicted.sort_values(ascending=False)
    selected = target[predicted_sorted.index][:SELECTED_WELLS]
    return INCOME_PER_BARREL * selected.sum() - BUDGET

In [20]:
def bootstrap(target, predicted):
    values = []
    for _ in range(SAMPLES):
        target_sub = target.sample(frac=(RESEARCH_WELLS / len(target)),
                                   replace=True, random_state=state)
        pred_sub = predicted[target_sub.index]
        income = profit(target_sub, pred_sub)
        values.append(income)
    values = pd.Series(values)
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    loss_risk = (values < 0).mean()
    return mean, (lower, upper), loss_risk

In [21]:
i = 1
for region in results_on_valid:
    mean, interval, loss_risk = bootstrap(region['target'],
                                          region['predicted'])
    print(f'Регион {i}:')
    print(f'Средняя прибыль: {mean}')
    print(f'95%-й доверительный интервал: {interval}')
    print(f'Риск убытков: {loss_risk:.2%}')
    print()
    i += 1

Регион 1:
Средняя прибыль: 449226331.6977277
95%-й доверительный интервал: (-74060657.26505223, 990322137.3103389)
Риск убытков: 5.30%

Регион 2:
Средняя прибыль: 496877990.0529474
95%-й доверительный интервал: (100097541.61165586, 920151026.8281939)
Риск убытков: 0.90%

Регион 3:
Средняя прибыль: 383213244.775705
95%-й доверительный интервал: (-168786451.18593073, 906615292.3482236)
Риск убытков: 8.30%



Наиболее привлекательным для разработки является второй регион. Здесь наибольшая средняя прибыль и наименьший риск убытков. Также 95%-й доверительный интервал выглядит лучше, чем для других регионов - он целиком расположен в области положительной прибыли.

## Выводы

На первом шаге были загружены данные. Предварительный анализ не показал каких-либо проблем с данными. Лишь были удалены неинформативные столбцы. 

Затем для каждого региона были подготовлены данные и обучена модель линейной регресии. На валидационной выборке были получены следующие результаты:

 - Регион 1:
   - Сред. запас: 92.46866515483404
   - RMSE модели: 37.50092951247967
 - Регион 2:
   - Сред. запас: 68.77217858390489
   - RMSE модели: 0.8899637258179826
 - Регион 3:
   - Сред. запас: 94.68357707424651
   - RMSE модели: 40.05955871704578
   
Далее был рассчитан необходимый объём сырья для безубыточной разработки скважины:
 - на регион - 22222.2 тыс.баррелей
 - на скважину - 111.1 тыс.баррелей

В среднем запасы в скважинах во всех регионах меньше, чем необходимо для безубыточной разработки новой скважины.

Наконец, с применением техники Bootstrap для каждого региона были рассчитаны прибыль и риски. С учётом полученных результатов наиболее привлекательным для разработки новых скважин оказался второй регион: здесь лучшие из трёх регионов показатели прибыли и наименьший риск убытков.